In [2]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-1.11.0-cp39-cp39-win_amd64.whl (157.9 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
pip install gluonnlp

Defaulting to user installation because normal site-packages is not writeable
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-win_amd64.whl size=454507 sha256=b55fff0f7f94502ecf3719112e27584e804f660601aa715a53e34be265fb6d81
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\47\17\70\b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install mxnet

Defaulting to user installation because normal site-packages is not writeable
  Using cached mxnet-1.7.0.post2-py2.py3-none-win_amd64.whl (33.1 MB)
  Using cached requests-2.18.4-py2.py3-none-any.whl (88 kB)
  Using cached numpy-1.16.6.zip (5.1 MB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Using cached urllib3-1.22-py2.py3-none-any.whl (132 kB)
  Using cached idna-2.6-py2.py3-none-any.whl (56 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-win_amd64.whl size=3712734 sha256=6f80b0065e9b95bba0aace61da51562075cb687bba39b05a797e47ce3d875507
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\ce\72\38\377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
Successfully built numpy
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
selenium 4.2.0 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.22 which is incompatible.
xarray 0.20.1 requires numpy>=1.18, but you have numpy 1.16.6 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have numpy 1.16.6 which is incompa

In [8]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable



In [ ]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")


dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

PATH = "/content/News.pt"

# 불러오기
device = torch.device("cuda")
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load(PATH))
model.to(device)
model.eval()


import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = [] # 텍스트 데이터를 담을 리스트
  emo_list = [] # 감성 값을 담을 리스트
  # for c in comment: # 모든 댓글
  commnetslist.append( [comment, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 부정, 긍정은 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

from numpy import genfromtxt
import numpy as np

from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen, re

########### 전일 거래량 20종목 가져오기 ############

def getStocks():    
    # 주식수 (20개)
    stockNum = 20

    # 해당 페이지가 주식 정보를 동적으로 만들어 selenium으로 데이터를 가져온다.
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    driver = webdriver.Chrome('chromedriver.exe', options=options)


    ####################### 종목명, 종목정보 링크(etf종목 구별할 때 사용) #################

    driver.get('https://finance.naver.com/sise/sise_quant.naver')

    time.sleep(5)
    bs = BeautifulSoup(driver.page_source, 'html.parser')

    # 테이블에서 PER|ROE|영업이익(etf,etn면 N/A) 위치
    index = 0 
    ths = bs.find(class_='type_2').find('tbody').find('tr').find_all('th')
    for i in range(len(ths)):
        if(ths[i].get_text() == 'PER' or ths[i].get_text() == 'ROE' or ths[i].get_text() == '영업이익'):
            index = i+1
            break
            
    
    # 테이블의 종목당 정보들 
    stocks = bs.find(class_='type_2').find('tbody').find_all('tr')

    result = []

    count = 0
    # count가 주식 수를 채울 때 까지 반복
    for i in range(len(stocks)):
        
        # 테이블에서 종목당 종목 정보만 가져오기
        stock = None
        try:
            no = stocks[i].find(class_="no").get_text()
            stock=stocks[i]
        except AttributeError:
            continue
        
        # PER|ROE|영업이익 N/A가 아니면 종목명 리스트에 넣기
        if(stock.select('td:nth-child({})'.format(index))[0].get_text() != 'N/A'):
            result.append(stock.find(class_="tltle").get_text()) 
            count += 1
        
        # 원하는 종목 만큼 (stockNum == 20)
        if(count>=stockNum):
            break

    driver.quit()

    return result

from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen, re

########### 전일 거래량 20종목 가져오기 ############

def getStocks():    
    # 주식수 (20개)
    stockNum = 20

    # 해당 페이지가 주식 정보를 동적으로 만들어 selenium으로 데이터를 가져온다.
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    driver = webdriver.Chrome('chromedriver.exe', options=options)


    ####################### 종목명, 종목정보 링크(etf종목 구별할 때 사용) #################

    driver.get('https://finance.naver.com/sise/sise_quant.naver')

    time.sleep(5)
    bs = BeautifulSoup(driver.page_source, 'html.parser')

    # 테이블에서 PER|ROE|영업이익(etf,etn면 N/A) 위치
    index = 0 
    ths = bs.find(class_='type_2').find('tbody').find('tr').find_all('th')
    for i in range(len(ths)):
        if(ths[i].get_text() == 'PER' or ths[i].get_text() == 'ROE' or ths[i].get_text() == '영업이익'):
            index = i+1
            break
            
    
    # 테이블의 종목당 정보들 
    stocks = bs.find(class_='type_2').find('tbody').find_all('tr')

    result = []

    count = 0
    # count가 주식 수를 채울 때 까지 반복
    for i in range(len(stocks)):
        
        # 테이블에서 종목당 종목 정보만 가져오기
        stock = None
        try:
            no = stocks[i].find(class_="no").get_text()
            stock=stocks[i]
        except AttributeError:
            continue
        
        # PER|ROE|영업이익 N/A가 아니면 종목명 리스트에 넣기
        if(stock.select('td:nth-child({})'.format(index))[0].get_text() != 'N/A'):
            result.append(stock.find(class_="tltle").get_text()) 
            count += 1
        
        # 원하는 종목 만큼 (stockNum == 20)
        if(count>=stockNum):
            break

    driver.quit()

    return result

#주식 거래량 상위 20위 
stocksList = getStocks()

#뉴스 헤드라인 크롤링 > 엑셀파일로 저장 
start(stocksList)


import csv

file = open('test1.csv', encoding='cp949')

type(file)

csvreader = csv.reader(file)

rows = []
names = []
for row in csvreader:
  if row[0] != '종목이름' and row[1]!= '기사제목':
    names.append(row[0])
    rows.append(row[1])
rows



for s in rows:
  # if len(s)>0 and s !='삼성전자':
  #   s=str(s)
  print(getSentimentValue(s, tok, max_len, batch_size, device),s)

count=0

for s in rows:
  if getSentimentValue(s, tok, max_len, batch_size, device)[0] == 1:
    count = count + 1

print(count)



ModuleNotFoundError: No module named 'kobert'